<a href="https://colab.research.google.com/github/SUDITI03/MLFT/blob/master/Firststepclear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import drive
drive.mount('/content/drive')

!pip install open3d
!pip install nuscenes-devkit --quiet


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [62]:

import numpy as np
import open3d as o3d
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from sklearn.linear_model import RANSACRegressor

# --- Load dataset ---
data_root = '/content/drive/MyDrive/nuscenes'
nusc = NuScenes(version='v1.0-mini', dataroot=data_root, verbose=True)
scene_indices = [0, 1]  # pick first 2 scenes

for scene_idx in scene_indices:
    scene = nusc.scene[scene_idx]
    first_sample_token = scene['first_sample_token']
    sample = nusc.get('sample', first_sample_token)

    # --- Load LiDAR points ---
    lidar_token = sample['data']['LIDAR_TOP']
    pc_path = nusc.get_sample_data_path(lidar_token)
    lidar_pc = LidarPointCloud.from_file(pc_path).points[:3, :].T  # Nx3

     # --- Ground segmentation ---
    X = lidar_pc[:, :2]  # xy
    y = lidar_pc[:, 2]   # z
    ransac = RANSACRegressor()
    ransac.fit(X, y)
    ground_z = ransac.predict(X)
    is_obstacle = y > ground_z + 0.2
    obstacle_points = lidar_pc[is_obstacle]
       # --- Semantic occupancy (sample annotations) ---
    ann_tokens = sample['anns']
    semantic_points = []
    for ann_token in ann_tokens:
        ann = nusc.get('sample_annotation', ann_token)
        category = ann['category_name']
        if category in ['vehicle.car', 'vehicle.truck', 'human.pedestrian']:
            box = nusc.get_box(ann_token)
            semantic_points.append(box.center)
    semantic_points = np.array(semantic_points)




Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.541 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [63]:
import open3d as o3d

for scene_idx in scene_indices:
    scene = nusc.scene[scene_idx]
    first_sample_token = scene['first_sample_token']
    sample = nusc.get('sample', first_sample_token)

    # --- Load LiDAR points ---
    lidar_token = sample['data']['LIDAR_TOP']
    pc_path = nusc.get_sample_data_path(lidar_token)
    lidar_pc = LidarPointCloud.from_file(pc_path).points[:3, :].T  # Nx3

    # --- Ground segmentation ---
    X = lidar_pc[:, :2]  # xy
    y = lidar_pc[:, 2]   # z
    ransac = RANSACRegressor()
    ransac.fit(X, y)
    ground_z = ransac.predict(X)
    is_obstacle = y > ground_z + 0.2
    obstacle_points = lidar_pc[is_obstacle]

    # --- Semantic occupancy (sample annotations) ---
    ann_tokens = sample['anns']
    semantic_points = []
    for ann_token in ann_tokens:
        ann = nusc.get('sample_annotation', ann_token)
        category = ann['category_name']
        if category in ['vehicle.car', 'vehicle.truck', 'human.pedestrian']:
            box = nusc.get_box(ann_token)
            semantic_points.append(box.center)
    semantic_points = np.array(semantic_points)

    # --- Open3D visualization ---
    pcd_all = o3d.geometry.PointCloud()
    pcd_all.points = o3d.utility.Vector3dVector(lidar_pc)
    pcd_all.paint_uniform_color([0.5, 0.5, 0.5])  # gray for all points

    pcd_obstacles = o3d.geometry.PointCloud()
    pcd_obstacles.points = o3d.utility.Vector3dVector(obstacle_points)
    pcd_obstacles.paint_uniform_color([1, 0, 0])  # red for obstacles

    if semantic_points.shape[0] > 0:
        pcd_semantic = o3d.geometry.PointCloud()
        pcd_semantic.points = o3d.utility.Vector3dVector(semantic_points)
        pcd_semantic.paint_uniform_color([0, 1, 0])  # green for cars/pedestrians

        o3d.visualization.draw_geometries([pcd_all, pcd_obstacles, pcd_semantic])
    else:
        o3d.visualization.draw_geometries([pcd_all, pcd_obstacles])


[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
# Install nuscenes devkit if not installed
!pip install nuscenes-devkit plotly scikit-learn

import numpy as np
import plotly.graph_objects as go
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from sklearn.linear_model import RANSACRegressor

# --- Load dataset ---
data_root = '/content/drive/MyDrive/nuscenes'  # change if needed
nusc = NuScenes(version='v1.0-mini', dataroot=data_root, verbose=True)
scene_indices = [0, 1]  # pick first 2 scenes

for scene_idx in scene_indices:
    scene = nusc.scene[scene_idx]
    print(f"\nProcessing Scene {scene_idx}: {scene['name']}")

    first_sample_token = scene['first_sample_token']
    sample = nusc.get('sample', first_sample_token)

    # --- Load LiDAR points ---
    lidar_token = sample['data']['LIDAR_TOP']
    pc_path = nusc.get_sample_data_path(lidar_token)
    lidar_pc = LidarPointCloud.from_file(pc_path).points[:3, :].T  # Nx3 array

    # --- Ground segmentation using RANSAC ---
    X = lidar_pc[:, :2]  # xy
    y = lidar_pc[:, 2]   # z
    ransac = RANSACRegressor(residual_threshold=0.2)
    ransac.fit(X, y)
    ground_z = ransac.predict(X)
    is_obstacle = y > ground_z + 0.2  # 20 cm above ground
    obstacle_points = lidar_pc[is_obstacle]

    # --- Semantic points from annotations ---
    semantic_points = []
    ann_tokens = sample['anns']
    for ann_token in ann_tokens:
        ann = nusc.get('sample_annotation', ann_token)
        category = ann['category_name']
        if category.startswith('vehicle.') or category.startswith('human.pedestrian'):
            # Get bounding box center
            box = nusc.get_box(ann_token)
            semantic_points.append(box.center)
    semantic_points = np.array(semantic_points)

    # --- Plot with Plotly ---
    fig = go.Figure()

    # Full LiDAR cloud (gray)
    fig.add_trace(go.Scatter3d(
        x=lidar_pc[:, 0], y=lidar_pc[:, 1], z=lidar_pc[:, 2],
        mode='markers',
        marker=dict(size=1, color='gray'),
        name='LiDAR Points'
    ))

    # Obstacle points (red)
    fig.add_trace(go.Scatter3d(
        x=obstacle_points[:, 0], y=obstacle_points[:, 1], z=obstacle_points[:, 2],
        mode='markers',
        marker=dict(size=2, color='red'),
        name='Obstacles / Non-drivable'
    ))

    # Semantic object centers (green)
    if semantic_points.shape[0] > 0:
        fig.add_trace(go.Scatter3d(
            x=semantic_points[:, 0], y=semantic_points[:, 1], z=semantic_points[:, 2],
            mode='markers',
            marker=dict(size=4, color='green'),
            name='Semantic Objects'
        ))

    fig.update_layout(
        title=f"Scene {scene_idx} - LiDAR + Obstacles + Semantic Objects",
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectmode='data'
        ),
        height=800
    )

    fig.show()


In [2]:
# ------------------------------------------------------------------------------
# Plotly 3D LiDAR + Semantic Boxes (nuScenes v1.0-mini)
# - Clean point cloud (ground vs obstacle-ish points)
# - True 3D oriented bounding boxes in LiDAR frame
# - Class-colored boxes + centers
# - ROI crop + downsample for speed
# ------------------------------------------------------------------------------

# pip install nuscenes-devkit plotly scikit-learn

import numpy as np
import plotly.graph_objects as go
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import BoxVisibility
from sklearn.linear_model import RANSACRegressor

# -----------------------
# CONFIG
# -----------------------
# --- Load dataset ---
data_root = '/content/drive/MyDrive/nuscenes'  # change if needed
nusc = NuScenes(version='v1.0-mini', dataroot=data_root, verbose=True)

  # <-- change this
SCENE_INDICES = [0, 1]                           # visualize first 2 scenes
ROI = dict(x_min=-5, x_max=50, y_min=-25, y_max=25)  # meters in LiDAR frame
DOWNSAMPLE_MAX_POINTS = 120_000                  # cap point count for Plotly
GROUND_THRESH = 0.20                              # meters above RANSAC ground = obstacle-ish

# Colors for categories
COLOR_MAP = {
    "vehicle": "red",
    "human.pedestrian": "lime",
    "barrier": "yellow",
    "trafficcone": "yellow",
    "construction": "orange",
    "other": "blue",
}

def cat_to_color(cat: str) -> str:
    if cat.startswith("vehicle."):
        return COLOR_MAP["vehicle"]
    if cat.startswith("human.pedestrian"):
        return COLOR_MAP["human.pedestrian"]
    if cat.startswith("barrier"):
        return COLOR_MAP["barrier"]
    if cat.startswith("trafficcone"):
        return COLOR_MAP["trafficcone"]
    if cat.startswith("construction"):
        return COLOR_MAP["construction"]
    return COLOR_MAP["other"]

def boxes_to_plotly_traces(boxes):
    """
    Convert nuScenes Box list (already in LiDAR frame when fetched via get_sample_data)
    into Plotly line traces for 3D oriented boxes + center markers.
    """
    line_traces = []
    center_traces = []
    # Edges of the 3D box (12 edges)
    edges = np.array([
        [0,1],[1,2],[2,3],[3,0],
        [4,5],[5,6],[6,7],[7,4],
        [0,4],[1,5],[2,6],[3,7]
    ])

    for box in boxes:
        cat = box.name  # e.g., 'vehicle.car'
        color = cat_to_color(cat)

        # 8 corners as 3x8 (x,y,z), convert to 8x3
        c = box.corners().T

        # Build a single line trace with gap segments for edges
        xs, ys, zs = [], [], []
        for e in edges:
            p1, p2 = c[e[0]], c[e[1]]
            xs += [p1[0], p2[0], None]
            ys += [p1[1], p2[1], None]
            zs += [p1[2], p2[2], None]

        line_traces.append(
            go.Scatter3d(
                x=xs, y=ys, z=zs,
                mode="lines",
                line=dict(width=3, color=color),
                name=cat,
                hoverinfo="name"
            )
        )

        # Center marker
        center = box.center  # (x,y,z)
        center_traces.append(
            go.Scatter3d(
                x=[center[0]], y=[center[1]], z=[center[2]],
                mode="markers",
                marker=dict(size=4, color=color, symbol="circle"),
                name=f"{cat} center",
                hoverinfo="name"
            )
        )
    return line_traces, center_traces

def ransac_ground_labels(points_xyz, residual_thresh=GROUND_THRESH):
    """
    Fit a simple ground plane z = f(x,y) with RANSACRegressor on (x,y)->z
    Returns boolean mask 'is_obstacleish' for points above the plane by threshold.
    """
    if points_xyz.shape[0] < 1000:
        return np.zeros(points_xyz.shape[0], dtype=bool)

    X = points_xyz[:, :2]  # x,y
    z = points_xyz[:, 2]   # z
    ransac = RANSACRegressor(residual_threshold=residual_thresh)
    ransac.fit(X, z)
    z_hat = ransac.predict(X)
    is_obs = z > (z_hat + residual_thresh)
    return is_obs

def roi_mask(points_xyz, roi=ROI):
    x, y = points_xyz[:, 0], points_xyz[:, 1]
    return (x >= roi["x_min"]) & (x <= roi["x_max"]) & (y >= roi["y_min"]) & (y <= roi["y_max"])

# -----------------------
# LOAD DATASET
# -----------------------


for scene_idx in SCENE_INDICES:
    scene = nusc.scene[scene_idx]
    first_sample_token = scene["first_sample_token"]
    sample = nusc.get("sample", first_sample_token)

    # LiDAR sample_data token
    lidar_token = sample["data"]["LIDAR_TOP"]

    # 1) Load point cloud (in LiDAR frame)
    pc_path = nusc.get_sample_data_path(lidar_token)
    lidar_points = LidarPointCloud.from_file(pc_path).points[:3, :].T  # Nx3

    # Apply ROI crop for clarity and speed
    mask_roi = roi_mask(lidar_points)
    lidar_points_roi = lidar_points[mask_roi]
    # Downsample if massive
    if lidar_points_roi.shape[0] > DOWNSAMPLE_MAX_POINTS:
        idx = np.random.choice(lidar_points_roi.shape[0], DOWNSAMPLE_MAX_POINTS, replace=False)
        lidar_points_roi = lidar_points_roi[idx]

    # 2) Ground vs obstacle-ish via RANSAC (within ROI)
    is_obstacleish = ransac_ground_labels(lidar_points_roi, residual_thresh=GROUND_THRESH)
    ground_pts = lidar_points_roi[~is_obstacleish]
    obst_pts = lidar_points_roi[is_obstacleish]

    # 3) Get oriented boxes ALREADY TRANSFORMED into LiDAR frame
    #    This avoids any coordinate mismatch.
    _, boxes, _ = nusc.get_sample_data(lidar_token, box_vis_level=BoxVisibility.ANY)
    box_lines, box_centers = boxes_to_plotly_traces(boxes)

    # -----------------------
    # PLOTLY FIGURE
    # -----------------------
    fig = go.Figure()

    # Ground points (light gray, tiny)
    if ground_pts.size:
        fig.add_trace(go.Scatter3d(
            x=ground_pts[:,0], y=ground_pts[:,1], z=ground_pts[:,2],
            mode="markers",
            marker=dict(size=1, opacity=0.8),
            name="Ground"
        ))

    # Obstacle-ish points (darker gray)
    if obst_pts.size:
        fig.add_trace(go.Scatter3d(
            x=obst_pts[:,0], y=obst_pts[:,1], z=obst_pts[:,2],
            mode="markers",
            marker=dict(size=1.5, opacity=0.9),
            name="Above ground (+)"
        ))

    # Boxes + centers
    for tr in box_lines + box_centers:
        fig.add_trace(tr)

    fig.update_layout(
        title=f"Scene {scene_idx}: LiDAR + Oriented 3D Boxes (LiDAR frame)",
        scene=dict(
            xaxis_title="X (m)",
            yaxis_title="Y (m)",
            zaxis_title="Z (m)",
            aspectmode="data"
        ),
        legend=dict(itemsizing="constant"),
        height=820
    )

    # Nice initial camera view (front-left, slightly above)
    fig.update_layout(
        scene_camera=dict(
            eye=dict(x=1.3, y=1.3, z=0.8)
        )
    )

    fig.show()


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.100 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
